#### Random Graph Generation

In [1]:
import numpy as np
import random
import pickle
import os

nV=[150]                      #nv is the number of vertices in the graph.  
nC=[15]                      #cv is the number of clients for each facility
num_graphs=5

problem='max-coverage'
modes=['skewed[1-10]']
da=['training','testing']
k=10
# mode='/skewed[1-6]'
# mode='/skewed[1,10]'
for nv in nV:
    for nc in nC:
        for mode in modes:
            os.system('mkdir'+' data/')
            os.system('mkdir'+' data/'+problem)
            os.system('mkdir'+' data/'+problem+'/'+mode)
            os.system('mkdir'+' data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc))
            os.system('mkdir'+' data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/testing')
            os.system('mkdir'+' data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/training')
            print(nv,nc,mode)
            for d in da:
                path='data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/'+d+'/'
                for graph in range(num_graphs):
                    adj=np.zeros([nv*nc,3])
                    new_pairs=[]
                    for i in range(1,nv+1):
                        y=np.sort(random.sample(range(1,nv+1), nc))
                        for j in (y):
                            new_pairs.append((i,j))
                    new_pairs=np.array(new_pairs)


                    adj[:,0]=new_pairs[:,0] #Set
                    adj[:,1]=new_pairs[:,1] #Element
                    #adj[:,2]= random.choices([1,10], weights = [3, 6], k = nc*nv)
                    #adj[:,2]np.random.randint(low = 1,high=6, size=nc*nv)
                    adj[:,2]= 1
                    #random.choices([1,2,3,4,5,6,7,8,9,10], weights = [1,1,2,2,3,3,4,4,5,5], k = nc*nv)
                    #adj[:,2]= random.choices([1,100], k = nc*nv)

                    os.system("mkdir "+path+str(graph)); pickle.dump([adj,nv],open(path+str(graph)+'/data','wb'))
                    np.savetxt(path+str(graph)+'/data.txt', adj, fmt='%10d' ,header=str(nv)+'   '+str(nv*nc))

150 15 skewed[1-10]


#### LP for feature computation

In [2]:
import xpress as xp
from termcolor import colored
import timeit
import numpy as np
import pickle
import random
def kmedian_xpress_solver_LP(set_path,c,n,k):
    print("Inside kmedian",n)
    S1 = timeit.default_timer()
    x = np.array ([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    print(colored("----------",'blue'))
    print(c)
    c1 = xp.Sum(y)<=k
    c2 = [x[i,j] - y[i] <=0 for i in range(n) for j in range(n)]
    c3 = [x[i,j]-c[i,j] <=0 for i in range(n) for j in range(n)]
    c4 = [xp.Sum(x[:,j]) <=1 for j in range(n)]
    #c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    #c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(x)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.maximize)
    try:
        p.addConstraint(c1,c2,c3,c4)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    y=p.getSolution(y)
    x=p.getSolution(x)
    E1 = timeit.default_timer()
    #################################################################################################
    median_indices=np.where(y>0.0)[0]
    print(colored('            Time: ','red'), E1 - S1) 
    print(colored('            objective value: ','red'),p.getSolution(profit))
    #print(colored('median indices: ','red'),median_indices+1,median_indices.shape)
     #------------------------------------------------------
    pickle.dump([median_indices,p.getSolution(profit),E1-S1],open(set_path+'/lp_inf','wb'))
    return median_indices,x,y

#### Computing LP based features

In [3]:
def features_related_LP(set_path,median_indices,x,y,c,n):
    #------------------lp_kmedian----------------------
    ind=np.zeros(n);ind[median_indices]=1
    pickle.dump(ind,open(set_path+'/lp_kmedian','wb'))
    pickle.dump(y,open(set_path+'/yf','wb'))
    #--------------------alfa_f-------------------------
    alfa_f=np.zeros(n)
    for i in range(n):
        alfa_f[i]= sum(x[:,i])
    pickle.dump(alfa_f,open(set_path+'/alfa_f','wb'))
    #---------------------µf-------------------------
    µf=np.zeros(n);
    for i in range(n):
        if alfa_f[i]!=0:
            µf[i]= sum(x[:,i]*c[:,i])/alfa_f[i]
    pickle.dump(µf,open(set_path+'/µf','wb'))
    #---------------------Vf,t(t=2,4,8)-------------------------
    T=[2,4,8]
    for t in T:
        V_ft=np.zeros(n);
        for i in range(n):
            V_ft[i]=len(np.where(c[:,i]<=(t*µf[i]))[0])
        pickle.dump(V_ft,open(set_path+'/V_f_'+str(t),'wb'))
    #---------------------Bf,t(t=2,4,8)-------------------------
    dav=np.zeros(n)
    for i in range(n):
        dav[i]=sum(c[:,i]*x[:,i])

    T=[2,4,8]
    for t in T:
        B_ft=np.zeros(n)
        for i in range(n):
            B_ft[i]=len(np.where(c[:,i]<=(t*dav[i]))[0])
        pickle.dump(B_ft,open(set_path+'/B_f_'+str(t),'wb'))
                    
    #----------------------------Yf----------------------------
    Yf=np.zeros(n)
    for i in range(n):
        Yf[i]=(len(np.where(x[:,i]!=0)[0]))
    pickle.dump(Yf,open(set_path+'/Yf','wb'))

#### Computing features not based on LP

In [4]:
def features_no_related_LP(Mypath,cost_mtr,nv):
    #--------------------------------------------------------
    degree=np.zeros(nv)
    for j in range(nv):
        s=np.where(cost_mtr[:,j]!=0)[0]
        degree[j]=sum(cost_mtr[s,j])
    pickle.dump(degree,open(Mypath+'/degree','wb'))
    #--------------------------------------------------------
    T=[2,4,8]
    for t in T:
        T_ft=np.zeros(nv);
        for i in range(nv):
            T_ft[i]=len(np.where(cost_mtr[:,i]<=t)[0])

        pickle.dump(T_ft,open(Mypath+'/T_f_'+str(t),'wb'))
    #--------------------------------------------------------
    min_c=np.zeros(nv)
    for i in range(nv):
        min_c[i]=min(cost_mtr[:,i])
    pickle.dump(min_c,open(Mypath+'/min_c','wb'))
    #---------------------------min_f-----------------------------
    min_f=np.zeros(nv)
    for i in range(nv):
        min_f[i]=min(cost_mtr[:,i])
    pickle.dump(min_f,open(Mypath+'/min_f','wb'))
    #----------------------------Yf,p=1.5----------------------------
    Yf=np.zeros(nv)
    for i in range(nv):
        Yf[i]=len(np.where(cost_mtr[:,i]<=(1.5*min_f[i]))[0])
    pickle.dump(Yf,open(Mypath+'/Yfp','wb'))
    #------------------------------------------------------------------------------------
    s=np.sort(random.sample(range(nv), int(nv/2)))
    s1=np.zeros(nv);s1[s]=1
    ss=[]
    for i in range(nv):
        if i not in s:
            ss.append(i)
    ss=np.array(ss);s2=np.zeros(nv);s2[ss]=1
    pickle.dump(s1,open(Mypath+'/f1','wb'))
    pickle.dump(s2,open(Mypath+'/f2','wb'))

#### Calling feature computation (LP-based and non LP-based) after computing LP

In [5]:
for Mode in modes:
    print(Mode,'--------------------------')
    for nv in nV:
        for nc in nC:
            print('------------------------------------------')
            for d in da:
                print('--- nV=',nv,' nC=',nc, ' mode=',d)
                for graph in range(num_graphs):
                    print('       data=',graph)
                    Mypath='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/'+d+'/'+str(graph)
                    data = pickle.load(open(Mypath+'/data','rb'))
                    nv= data[1]; data=data[0]
                    data=data.astype(int)
                    print("Data: ",data)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1
                    cost_mtr=np.zeros([nv,nv])
                    for i in range (len(data)):
                        cost_mtr[data[i,0],data[i,1]]=data[i,2]
                    #print("Calling kmedian with ",Mypath,data.shape,cost_mtr.shape,nv,k)
                    median_indices,x,y=kmedian_xpress_solver_LP(Mypath,cost_mtr,nv,k)
                    features_related_LP(Mypath,median_indices,x,y,cost_mtr,nv)
                    features_no_related_LP(Mypath,cost_mtr,nv)

skewed[1-10] --------------------------
------------------------------------------
--- nV= 150  nC= 15  mode= training
       data= 0
Data:  [[  1   8   1]
 [  1  20   1]
 [  1  24   1]
 ...
 [150 127   1]
 [150 130   1]
 [150 132   1]]
Inside kmedian 150
(150, 150) (150,)
----------
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 0:03:06, Aug 15, 2021
Heap usage: 16MB (peak 16MB, 1204KB system)
Maximizing LP noname
Original problem has:
     45151 rows        22650 cols        90150 elements
Presolved problem has:
      2401 rows         2400 cols         6900 elements
Presolve finished in 0 seconds
Heap usage: 14MB (peak 33MB, 1206KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1

  4312        144.448851      P      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  4312 simplex iterations in 0s

Final objective                       : 1.444488513764246e+02
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
            Time:  0.9126540021970868
            objective value:  144.44885137642464

--- nV= 150  nC= 15  mode= testing
       data= 0
Data:  [[  1   9   1]
 [  1  30   1]
 [  1  41   1]
 ...
 [150 124   1]
 [150 127   1]
 [150 139   1]]
Inside kmedian 150
(150, 150) (150,)
----------
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 0:03:11, Aug 15, 2021
Heap usage: 16MB (peak 16MB, 1242KB system)
Maximizing LP noname
Original problem 

  4041        144.118837      P      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  4041 simplex iterations in 0s

Final objective                       : 1.441188366110085e+02
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
            Time:  0.8299416587688029
            objective value:  144.1188366110085



#### Run ILP to get labelled training data and evaluation on test data

In [6]:
import xpress as xp
from termcolor import colored
import timeit
import math
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import networkx as nx

def floyd_warshall(G,nV):
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))

    # Adding vertices individually
    for k in range(nV):
        for i in range(nV):
            for j in range(nV):
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
    return (distance)

def kmedian_xpress_solver_ILP(c,n,k):
    x = np.array ([xp.var (vartype = xp.binary) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    print(colored("----------",'blue'))
    print(c)
    c1=[xp.Sum(y)==k]
    c2 = [x[i,j] - y[i] <=0 for i in range(n) for j in range(n)]
    c3 = [x[i,j]-c[i,j] <=0 for i in range(n) for j in range(n)]
    c4 = [xp.Sum(x[:,j]) <=1 for j in range(n)]
    #c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    #c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(x)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.maximize)
    try:
        p.addConstraint(c1,c2,c3,c4)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    

    median_indices=np.where(p.getSolution(y)>=.5)[0]
    return median_indices,p.getSolution(profit)


for nv in nV:
    for nc in nC:
        for mode in modes:
            for d in da:
                path='data/'+problem+'/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/'+d+'/'
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    S = timeit.default_timer()
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))
                    #print(data[0],data[1],data[2])
                    n = data[1]
                    data=data[0]
                    data=data.astype(int)
                    print("Data: ",data)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1
                    cost_mtr=np.zeros([n,n])
                    for i in range (len(data)):
                        cost_mtr[data[i,0],data[i,1]]=data[i,2]

                    labels=kmedian_xpress_solver_ILP(cost_mtr,n,k)

                    E = timeit.default_timer() 
                    pickle.dump([labels,E-S],open(set_path+'/points_inf_xp','wb'))

                    print(data[1],data[0].shape)
                    print(colored('objective value: ','red'),labels[1])
                    print(E - S) 
                    print(colored('median indices: ','red'),labels[0]+1,len(labels[0]))

.................................................... 0
Data:  [[  1   8   1]
 [  1  20   1]
 [  1  24   1]
 ...
 [150 127   1]
 [150 130   1]
 [150 132   1]]
(150, 150) (150,)
----------
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 0:03:16, Aug 15, 2021
Heap usage: 16MB (peak 16MB, 1253KB system)
Maximizing MILP noname
Original problem has:
     45151 rows        22650 cols        90150 elements     22650 globals
Presolved problem has:
      2401 rows         2400 cols         6900 elements      2400 globals
Presolve finished in 0 seconds
Heap usage: 19MB (peak 40MB, 1255KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max]

 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
1351994        145.176150      P      0     0        .000000    29
Optimal solution found
Dual solved problem
  1351994 simplex iterations in 29s

Final objective                       : 1.451761499710711e+02
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
 
Starting root cutting & heuristics
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
   1  K    118.000000   142.901430      3      0      0   17.43%    1527     30
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 8
Cut elements in the matrix : 15746
Will try to keep branch and bound tree memory usage below 65.5GB
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to 64 tasks.
Heap usage: 24MB (peak 482MB, 2873KB system)
Heap usage: 24MB 

    9528   120.000000   142.901430      4   6215     72   16.03%     605     76
    9629   120.000000   142.901430      4   6299     17   16.03%     600     77
    9729   120.000000   142.901430      4   6370     14   16.03%     757     77
    9834   120.000000   142.901430      4   6576     34   16.03%     651     77
    9937   120.000000   142.901430      4   6678     44   16.03%     598     77
   10040   120.000000   142.901430      4   6797     44   16.03%     464     77
   10146   120.000000   142.901430      4   6947     93   16.03%     532     77
   10247   120.000000   142.901430      4   7012     39   16.03%     710     77
   10351   120.000000   142.901430      4   7191     52   16.03%     402     77
q  10380   121.000000   142.901430      5   7203     25   15.33%       0     77
   10818   121.000000   142.901430      5   6869      1   15.33%       1     77
   10927   121.000000   142.901430      5   7019     30   15.33%     481     77
   11031   121.000000   142.901430      

   79340   125.000000   139.386223      7  45672     48   10.32%     718    139
   80341   125.000000   139.125265      7  46325     33   10.15%     878    140
   81343   125.000000   139.125265      7  46876     53   10.15%     740    141
Heap usage: 1094MB (peak 1094MB, 18MB system)
B&B tree size: 134MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   82343   125.000000   139.125265      7  47511     20   10.15%     658    142
   83344   125.000000   138.966887      7  48074     61   10.05%     722    143
   84348   125.000000   138.966887      7  48599     40   10.05%     588    144
   85348   125.000000   138.828170      7  49214     26    9.96%     953    145
   86352   125.000000   138.828170      7  49814     65    9.96%     692    146
   87352   125.000000   138.828170      7  50386     14    9.96%     772    147
   88352   125.000000   138.825964      7  51006     18    9.96%     674    147
   89358   125.000000   138.825964      7  51

FICO Xpress v8.10.0, Hyper, solve started 0:18:48, Aug 15, 2021
Heap usage: 16MB (peak 16MB, 1478KB system)
Maximizing MILP noname
Original problem has:
     45151 rows        22650 cols        90150 elements     22650 globals
Presolved problem has:
      2401 rows         2400 cols         6900 elements      2400 globals
Presolve finished in 0 seconds
Heap usage: 19MB (peak 40MB, 1479KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 65.0GB
 *** Heuristic solution found:    98.000000      Time: 0 ***
Starting concurrent solve with dual, primal and barrier (30 threads)

                           Concurrent-Solve,   0s
        

...

   90389   124.000000   135.869347      6  58869     61    8.74%     586    126
   91398   124.000000   135.869347      6  59406     46    8.74%     534    127
   92404   124.000000   135.869347      6  59897     29    8.74%     618    128
   93404   124.000000   135.838333      6  60530     44    8.72%     600    128
Heap usage: 1123MB (peak 1123MB, 18MB system)
B&B tree size: 186MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   94404   124.000000   135.838333      6  60996     32    8.72%     557    129
   95412   124.000000   135.838333      6  61618     41    8.72%     582    130
   96424   124.000000   135.816609      6  62131     57    8.70%     465    131
   97424   124.000000   135.816609      6  62679     38    8.70%     529    132
   98424   124.000000   135.816609      6  63278     50    8.70%     738    132
   99425   124.000000   135.621583      6  63818     26    8.57%     688    133
  100433   124.000000   135.621583      6  64

      40   113.000000   142.187746      4     36      8   20.53%     493      7
      50   113.000000   142.187746      4     44      9   20.53%     945      7
      60   113.000000   141.898772      4     54      8   20.37%    1314      7
      70   113.000000   141.898772      4     60      7   20.37%     954      8
      80   113.000000   141.898772      4     67      9   20.37%    1142      8
      90   113.000000   141.898772      4    102     11   20.37%     988      8
     100   113.000000   141.756329      4    124     10   20.29%     963      8
Heap usage: 263MB (peak 263MB, 6864KB system)
B&B tree size: 475k total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
     200   113.000000   141.420063      4    361     23   20.10%     520      9
a    238   120.000000   141.420063      5    445     20   15.15%       0     10
     300   120.000000   141.420063      5    568     21   15.15%     898     11
     400   120.000000   141.420063      5    7

    5283   124.000000   141.420063      7   1963     40   12.32%     747     53
    5383   124.000000   141.420063      7   2066     44   12.32%     783     54
    5483   124.000000   141.420063      7   2129     19   12.32%     883     55
    5584   124.000000   141.420063      7   2206     42   12.32%     626     55
    5684   124.000000   141.420063      7   2278     56   12.32%     843     56
    5784   124.000000   141.420063      7   2326     46   12.32%     775     57
    5884   124.000000   141.420063      7   2349     55   12.32%     742     58
    5984   124.000000   141.420063      7   2399     59   12.32%     800     58
    6084   124.000000   141.420063      7   2426     37   12.32%     859     59
    6184   124.000000   141.420063      7   2546     54   12.32%     735     60
    6284   124.000000   141.420063      7   2670     37   12.32%    1103     60
    6384   124.000000   141.420063      7   2775     35   12.32%     892     61
    6484   124.000000   141.420063      

   31165   124.000000   141.197142      7  22006     39   12.18%     712     87
   32165   124.000000   141.197142      7  22622     49   12.18%     572     87
   33171   124.000000   141.147919      7  23365     54   12.15%     584     88
Heap usage: 1030MB (peak 1030MB, 18MB system)
B&B tree size: 66MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   34177   124.000000   141.147919      7  23998     75   12.15%     656     89
   35177   124.000000   141.147919      7  24748     43   12.15%    1039     90
   36178   124.000000   140.237650      7  25350     61   11.58%     678     91
   37180   124.000000   140.237650      7  26089     34   11.58%     829     92
   38191   124.000000   140.237650      7  26767     39   11.58%     520     93
   39197   124.000000   140.237650      7  27417     60   11.58%     585     94
   40199   124.000000   140.217026      7  28056     15   11.57%     699     95
   41201   124.000000   140.217026      7  287

  315218   126.000000   128.128372      9  34368     24    1.66%     609    453
  325218   126.000000   127.892384      9  26586     34    1.48%     674    496
  335218   126.000000   127.609610      9  18356     25    1.26%     617    511
  345218   126.000000   127.348984      9   9682     33    1.06%     706    521
  355218   126.000000   127.038058      9    234     59    0.82%     690    529
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.260000000000001e+02
Final MIP bound                       : 1.260000000000001e+02
  Solution time / primaldual integral :       530s/  5.705758%
  Number of solutions found / nodes   :         9 /    355451
  Max primal violation      (abs/rel) : 2.887e-13 / 2.887e-13
  Max integer violation     (abs    ) : 2.887e-13

[ 0 20  1] (3,)
objective value:  126.00000000000014
530.3620485523716
median indices:  [  2  32  34  56  66  83  97 100 140 146] 10


#### Load libraries to run Classification model

In [1]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,precision_recall_curve, confusion_matrix, accuracy_score
import xgboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve
from sklearn import svm

#### Load features for classification

In [2]:
def feature_loader(set_path): 
    f0=pickle.load(open(set_path+'/alfa_f','rb'))
    f1=pickle.load(open(set_path+'/B_f_2','rb'))
    f2=pickle.load(open(set_path+'/B_f_4','rb'))
    f3=pickle.load(open(set_path+'/B_f_8','rb'))
    f4=pickle.load(open(set_path+'/lp_kmedian','rb'))
    f5=pickle.load(open(set_path+'/min_f','rb'))
    f6=pickle.load(open(set_path+'/V_f_2','rb'))
    f7=pickle.load(open(set_path+'/V_f_4','rb'))
    f8=pickle.load(open(set_path+'/V_f_8','rb'))
    f9=pickle.load(open(set_path+'/yf','rb'))
    f10=pickle.load(open(set_path+'/Yf','rb'))
    f11=pickle.load(open(set_path+'/µf','rb'))
    f12=pickle.load(open(set_path+'/Yfp','rb'))
#     f13=pickle.load(open(set_path+'/degree','rb'))
    f14=pickle.load(open(set_path+'/T_f_2','rb'))
    f15=pickle.load(open(set_path+'/T_f_4','rb'))
    f16=pickle.load(open(set_path+'/T_f_8','rb'))
#     f17=pickle.load(open(set_path+'/min_c','rb'))
#     f18=pickle.load(open(set_path+'/f1','rb'))
#     f19=pickle.load(open(set_path+'/f2','rb'))
#     f20=pickle.load(open(set_path+'/lp_prime','rb'))
#     f21=pickle.load(open(set_path+'/NLP1','rb'))
    #f22=pickle.load(open(set_path+'/NILP','rb'))
    #f23= pickle.load(open(set_path+'/ga_ind','rb'))
    
    features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:],f12[:],f14[:],f15[:],f16[:]]);features=features.transpose()
    #features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:]]);features=features.transpose()
    #features=np.vstack([f4[:],f10[:],f7[:],f2[:]]);features=features.transpose()
    
    
    return(features)

In [4]:
nV=[150]                      #nv is the number of vertices in the graph.  
nC=[15]                      #cv is the number of clients for each facility
num_graphs=5

problem='max-coverage'
modes=['skewed[1-10]']
da=['training','testing']
k=10

#### Load ground truth for classification

In [5]:
path='data/'+problem+'/'+modes[0]+'/V='+str(nV[0])+'-C='+str(nC[0])+'/training/0'
y=pickle.load(open(path+'/points_inf_xp','rb'))
y_true=np.zeros(nV[0]);y_true[y[0][0]]=1
features=feature_loader(path)


for Mode in modes:
    for nv in nV:
        for nc in nC:
            for i in range(1,num_graphs):
                Mypath='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/training/'+str(i)
                y=pickle.load(open(Mypath+'/points_inf_xp','rb'))
                y_temp=np.zeros(nV[0]);y_temp[y[0][0]]=1
                y_true=np.hstack([y_true,y_temp])
                f_temp=feature_loader(Mypath)
                features=np.vstack([features,f_temp])
print(len(y_true),len(features))


750 750


#### Normalisation and Learning Classification Model

In [6]:
sc = StandardScaler()
features = sc.fit_transform(features)


clf = RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=10, random_state=0,class_weight="balanced")
# clf = xgboost.XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0, class_weight='balanced')
# clf=AdaBoostClassifier(n_estimators=100, random_state=0)
# clf=svm.SVC(cache_size=200, class_weight='balanced',random_state=None)

clf.fit(features,y_true)

print(colored('Feature importance:','blue'))
print(clf.feature_importances_)

Feature importance:
[0.10681747 0.00356637 0.         0.         0.07301832 0.
 0.         0.         0.         0.4396239  0.15222902 0.
 0.22474492 0.         0.         0.        ]


#### Output the learnt model to a file

In [7]:
for Mode in modes:
    for nv in nV:
        for nc in nC:
            for i in range(num_graphs):
                test_path = 'data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/testing/'+str(i)

                test_features=feature_loader(test_path)
                test_features = sc.transform(test_features)

                y_pred = clf.predict(test_features)

                probs= clf.predict_proba(test_features)

                pickle.dump(np.where(y_pred==1)[0],open(test_path+'/pruning_points_'+str(i),'wb'))
                pickle.dump(probs[:,1],open(test_path+'/probs','wb'))
                pickle.dump(np.where(y_pred==1)[0],open(test_path+'/probs_xgb','wb'))


                print(len(np.where(y_pred==1)[0]))

18
15
16
12
11


#### Compute the ILP on pruned data set (pruning based on different thresholds)

In [8]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_pruned(c,n,num_fac,k):
    x = np.array ([xp.var (vartype = xp.binary) for i in range(num_fac*n)]).reshape (num_fac,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(num_fac)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    print(colored("----------",'blue'))
    print(c)
    c1=[xp.Sum(y)==k]
    c2 = [x[i,j] - y[i] <=0 for i in range(num_fac) for j in range(n)]
    c3 = [x[i,j]-c[i,j] <=0 for i in range(num_fac) for j in range(n)]
    c4 = [xp.Sum(x[:,j]) <=1 for j in range(n)]
    #c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    #c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(x)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.maximize)
    try:
        p.addConstraint(c1,c2,c3,c4)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)
    
    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            path='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/testing/'
            thre = [max(int(nv/10),1),max(int(nv/5),1),max(int(3*nv/10),1),max(int(4*nv/10),1),int(nv/2),int(nv)]
            value_n=np.zeros(len(thre))
            Time=np.zeros(len(thre))
            for m in range(num_graphs):
                print(colored('....................................................','blue'),m)
                set_path=path+str(m)
                data = pickle.load(open(set_path+'/data','rb'))

                probs=pickle.load(open(set_path+'/probs','rb'))
                ind=np.argsort(probs)


                n=data[1];
                data=data[0].astype(int)
                data[:,0]=data[:,0]-1
                data[:,1]=data[:,1]-1

                for th_index,th in enumerate(thre):
                    ind_last_thre = ind[len(ind)-th:]
                    num_fac = len(ind_last_thre)

                    c=np.zeros([len(ind_last_thre),n])
                    for i in range(n):
                        for j in range(len(ind_last_thre)):
                                c[j,i]=0

                    for i in range (len(ind_last_thre)):
                        mp=(np.where(data[:,0]==ind_last_thre[i])[0])
                        c[i,data[mp,1]]=data[mp,2]

                    #print(colored("--------------------",'blue'))
                    #print(data,n,c,num_fac,k)
                    #print(colored("--------------------",'blue'))
                    S=timer()
                    median_indices,value=kmedian_xpress_solver_pruned(c,n,len(ind_last_thre),k)
                    E=timer()
                    print(colored('pruned centers:','red'),np.sort(ind_last_thre)+1)
                    print(colored('selected centers:','red'),np.sort(ind_last_thre[(median_indices)])+1,len(median_indices))
                    print(colored('Objective Value:','red'),value)
                    print(E-S)

                    s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                    s=s[0][1]
                    value_n[th_index]=value/s
                    Time[th_index]=E-S
                    #print(s,value_n[thi],value)
                print(colored("Value_n: " +str(value_n),'red'))
                print(colored("Time: "+str(Time),'red'))

.................................................... 0
(15, 150) (15,)
----------
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
FICO Xpress v8.10.0, Hyper, solve started 19:01:09, Aug 15, 2021
Heap usage: 1873KB (peak 1873KB, 811KB system)
Maximizing MILP noname
Original problem has:
      4651 rows         2265 cols         9015 elements      2265 globals
Presolved problem has:
       230 rows          178 cols          504 elements       178 globals
Presolve finished in 0 seconds
Heap usage: 2125KB (peak 4323KB, 812KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  8.00e+00]
Autoscaling applied standard scaling

Symmetric


 Concurrent-Solve,   0s
            Dual        
    objective   sum inf 
 P  139.27138   .0000000
------- optimal --------
Concurrent statistics:
      Dual: 1461 simplex iterations, 0.04s
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
  1461        139.271381      P      0     0        .000000     0
Dual solved problem
  1461 simplex iterations in 0s

Final objective                       : 1.392713806517083e+02
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
 
Starting root cutting & heuristics
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
a          112.000000   139.271381      2                 19.58%       0      0
   1  K    112.000000   138.595735      2     10      0   19.19%     659      0
a          114.000000   138.340275      3                 17.59%       0

    5563   122.000000   129.878112      8     75     13    6.07%     530      5
    5664   122.000000   129.689170      8     74     13    5.93%     387      5
    5765   122.000000   129.209979      8     50     11    5.58%     387      5
    5865   122.000000   129.209979      8     38     11    5.58%     295      5
Heap usage: 428MB (peak 428MB, 74MB system)
B&B tree size: 1.4MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    5965   122.000000   128.409833      8     35     13    4.99%     434      5
    6065   122.000000   127.705518      8     38     16    4.47%     489      5
    6165   122.000000   127.425822      8     22     13    4.26%     331      5
    6266   122.000000   124.283019      8      3     13    1.84%     379      5
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.219999999999724e+02
Final MIP bound                       : 1.220000000000000e+02
  Solution time / primaldual integral 

    2568   122.000000   137.516864      8    919     40   11.28%     404      6
    2668   122.000000   137.516864      8    917     35   11.28%     389      6
    2778   122.000000   137.516864      8    935      1   11.28%       1      6
    2880   122.000000   137.516864      8    954      1   11.28%       1      6
    2984   122.000000   137.516864      8    972     15   11.28%     440      6
Resetting tree to root.

Performing root presolve...

Reduced problem has:    1061 rows     960 columns      7912 elements
Presolve dropped   :       0 rows       0 columns         0 elements
Will try to keep branch and bound tree memory usage below 64.4GB
Crash basis containing 60 structural columns created
 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
583738        141.126855      P      0     0        .000000     6
Optimal solution found
Dual solved problem
  583738 simplex iterations in 6s

Final objective                       : 1.411268554813396e+02
  Max primal vio

   2  K    112.000000   142.308033      4      3      1   21.30%    1052      0
   3  K    112.000000   142.210443      4      2      3   21.24%    1043      0
q          115.000000   142.176765      5                 19.11%       0      0
   4  K    115.000000   142.176765      5      2      2   19.11%    1071      0
   5  K    115.000000   142.036753      5      2      0   19.04%    1033      1
   6  K    115.000000   142.028041      5      3      3   19.03%    1005      1
   7  K    115.000000   142.027840      5      1      3   19.03%    1027      1
   8  K    115.000000   142.027840      5      0      0   19.03%    1035      2
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 7
Cut elements in the matrix : 5420
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to 64 tasks.
Heap usage: 16MB (peak 32MB, 2372KB system)
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
       1   115.000000   

...

    7733   123.000000   133.923980      6   1837     23    8.16%     415     27
    7835   123.000000   133.923980      6   1867     12    8.16%     508     27
    7935   123.000000   133.923980      6   1872     31    8.16%     544     27
    8037   123.000000   133.923980      6   1869     17    8.16%     489     27
a   8123   124.000000   133.923980      7   1872     16    7.41%       0     27
    8137   124.000000   133.923980      7   1874     21    7.41%     487     27
    8394   124.000000   133.923980      7   1630      1    7.41%       1     27
    8495   124.000000   133.923980      7   1642     21    7.41%     449     27
    8599   124.000000   133.923980      7   1642     16    7.41%     454     27
    8700   124.000000   133.923980      7   1654     21    7.41%     554     27
    8800   124.000000   133.923980      7   1661     19    7.41%     518     27
a   8820   125.000000   133.923980      8   1652      9    6.66%       0     27
    9429   125.000000   133.923980      

       4   113.000000   143.201913      4      5      4   21.09%    1433      7
       5   113.000000   143.201913      4      5      4   21.09%    1262      7
       6   113.000000   143.201913      4      6      4   21.09%     943      7
       7   113.000000   142.777649      4      7      4   20.86%    1235      7
       8   113.000000   142.777649      4      8      5   20.86%    1435      7
       9   113.000000   142.510553      4      9      5   20.71%    1253      7
      10   113.000000   142.510553      4     10      5   20.71%     955      7
      20   113.000000   142.510553      4     19      6   20.71%     708      7
      30   113.000000   141.971366      4     29      7   20.41%     933      8
      40   113.000000   141.971366      4     36      7   20.41%     723      8
      50   113.000000   141.971366      4     45      7   20.41%     969      8
      60   113.000000   141.971366      4     53      8   20.41%     449      8
      70   113.000000   141.955045      

    5176   123.000000   141.741391      7   1737     32   13.22%    1022     56
    5276   123.000000   141.741391      7   1880     33   13.22%     846     58
    5376   123.000000   141.741391      7   2028     31   13.22%     626     59
    5488   123.000000   141.741391      7   2162     36   13.22%     621     60
    5588   123.000000   141.741391      7   2324     38   13.22%     694     61
    5688   123.000000   141.741391      7   2480     43   13.22%     628     62
    5788   123.000000   141.741391      7   2618     51   13.22%     784     64
Heap usage: 955MB (peak 955MB, 18MB system)
B&B tree size: 3.9MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    5888   123.000000   141.741391      7   2679     46   13.22%     605     65
    5988   123.000000   141.741391      7   2715     21   13.22%     925     65
    6088   123.000000   141.741391      7   2823     26   13.22%     797     66
    6188   123.000000   141.741391      7   285

   22780   123.000000   141.380402      7  16806     41   13.00%     677     91
   23785   123.000000   138.790430      7  17326     51   11.38%     717     92
   24788   123.000000   138.790430      7  18212     44   11.38%     718     93
   25788   123.000000   138.790430      7  18848     61   11.38%     814     94
   26792   123.000000   138.633159      7  19576     44   11.28%     764     95
   27796   123.000000   138.633159      7  20286     37   11.28%     806     96
   28799   123.000000   138.476437      7  21093     27   11.18%     896     97
   29799   123.000000   138.476437      7  21706     24   11.18%     868     98
   30802   123.000000   138.293462      7  22441     25   11.06%     452    100
   31803   123.000000   138.293462      7  23190     27   11.06%     787    101
   32805   123.000000   138.293462      7  23737     32   11.06%     875    102
   33806   123.000000   138.258368      7  24593     39   11.04%     760    103
   34812   123.000000   138.258368      

  234048   124.000000   130.660336      8 117140     28    5.10%     571    335
  244052   124.000000   130.539547      8 119509     23    5.01%     440    347
* 248992   125.000000   130.514093      9 120644     42    4.22%       0    353
  265642   125.000000   130.514093      9 104016      1    4.22%       1    353
  275644   125.000000   130.255289      9 104024     19    4.03%     453    368
  285651   125.000000   130.073841      9 103671     38    3.90%     595    385
a 294775   126.000000   129.922405     10 102906     19    3.02%       0    394
  332606   126.000000   129.922405     10  65079      1    3.02%       1    395
  342606   126.000000   129.342016     10  59346     27    2.58%     456    406
  352607   126.000000   128.708271     10  51029     52    2.10%     718    415
  362607   126.000000   128.292977     10  41826     22    1.79%     664    423
  372607   126.000000   127.901645     10  32429     54    1.49%     730    428
  382607   126.000000   127.606543     1

#### Add a constraint that at least 80% of nodes to be taken from pruned set

In [9]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_constrained(c,n,fac_index,k):
    x = np.array ([xp.var (vartype = xp.binary) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(n)])

    c1=[xp.Sum(y)==k]
    c2 = [x[i,j] - y[i] <=0 for i in range(n) for j in range(n)]
    c3 = [x[i,j]-c[i,j] <=0 for i in range(n) for j in range(n)]
    c4 = [xp.Sum(x[:,j]) <=1 for j in range(n)]
    S=range(n)
    fac_inverse = [i for i in S if i not in fac_index]
    print(colored(fac_inverse,'magenta'))
    if len(fac_inverse) > 0:
        c5=[xp.Sum(y[fac_inverse])<=int(0.2*k)]
    else:
        c5=[xp.Sum(y[0])<=1]
    
    
    profit=xp.Sum(x)

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.maximize)
    try:
        p.addConstraint(c1,c2,c3,c4,c5)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    
  
    

   
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
 
    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:        
            path='data/'+problem+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/testing/'
            thre = [max(int(nv/10),1),max(int(nv/5),1),max(int(3*nv/10),1),max(int(4*nv/10),1),int(nv/2),int(nv)]
            value_n=np.zeros(len(thre))
            Time=np.zeros(len(thre))
            for m in range(num_graphs):
                print(colored('....................................................','blue'),m)
                set_path=path+str(m)
                data = pickle.load(open(set_path+'/data','rb'))

                probs=pickle.load(open(set_path+'/probs','rb'))
                ind=np.argsort(probs)


                n=data[1];
                data=data[0].astype(int)
                data[:,0]=data[:,0]-1
                data[:,1]=data[:,1]-1

                for th_index,th in enumerate(thre):
                    ind_last_thre = ind[len(ind)-th:]

                    cost_mtr=np.zeros([n,n])
                    for i in range (len(data)):
                        cost_mtr[data[i,0],data[i,1]]=data[i,2]

                    #print(colored("--------------------",'blue'))
                    #print(data,n,c,num_fac,k)
                    #print(colored("--------------------",'blue'))
                    S=timer()
                    median_indices,value=kmedian_xpress_solver_constrained(cost_mtr,n,ind_last_thre,k)
                    E=timer()
                    print(colored('pruned centers:','red'),np.sort(ind_last_thre)+1)
                    print(colored('selected centers:','red'),np.sort(median_indices)+1,len(median_indices))
                    print(colored('Objective Value:','red'),value)
                    print(E-S)

                    s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                    s=s[0][1]
                    value_n[th_index]=value/s
                    Time[th_index]=E-S
                    #print(s,value_n[thi],value)
                print(colored("Value_n: " +str(value_n),'red'))
                print(colored("Time: "+str(Time),'red'))

.................................................... 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 69, 70, 71, 72, 73, 74, 75, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
FICO Xpress v8.10.0, Hyper, solve started 19:39:48, Aug 15, 2021
Heap usage: 16MB (peak 16MB, 1531KB system)
Maximizing MILP noname
Original problem has:
     45152 rows        22650 cols        90285 elements     22650 globals
Presolved problem has:
      2402 rows         2400 cols         6915 elements      2400 globals
Presolve finished in 0 seconds
Heap usage:

   2  K    115.000000   138.696024      2      3      1   17.08%     929      0
   3  K    115.000000   138.607508      2      3      2   17.03%     908      0
a          116.000000   138.565777      3                 16.29%       0      1
   4  K    116.000000   138.565777      3      3      4   16.29%     907      1
   5  K    116.000000   138.491590      3      3      1   16.24%     888      1
   6  K    116.000000   138.479963      3      3      2   16.23%     924      1
   7  K    116.000000   138.471831      3      3      2   16.23%     878      1
   8  K    116.000000   138.471831      3      0      2   16.23%     887      2
Heuristic search started
R          118.000000   138.471831      4                 14.78%       0      3
R          121.000000   138.471831      5                 12.62%       0      3
Heuristic search stopped
 
Cuts in the matrix         : 9
Cut elements in the matrix : 12476
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to

    6710   123.000000   131.335488      6    117     21    6.35%     516     23
    6810   123.000000   131.333562      6    124     12    6.35%     504     23
    6910   123.000000   131.333562      6    142     16    6.35%     517     23
    7010   123.000000   131.333562      6    150     13    6.35%     438     24
    7110   123.000000   131.333562      6    142     11    6.35%     492     24
    7210   123.000000   130.549330      6    108     11    5.78%     219     24
    7310   123.000000   130.549330      6     63     15    5.78%     486     24
    7414   123.000000   130.549330      6     38     15    5.78%     357     25
    7515   123.000000   130.549330      6     39     16    5.78%     378     25
    7616   123.000000   129.777469      6     21     12    5.22%     334     25
    7716   123.000000   127.153779      6     10     18    3.27%     434     25
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.230000000000000e+02
Final MIP bou

    1501   115.000000   138.264494      5   1454     23   16.83%     534     26
k   1558   116.000000   138.264494      6   1462     24   16.10%       0     27
    1610   116.000000   138.264494      6   1415      1   16.10%       1     27
    1711   116.000000   138.264494      6   1426     35   16.10%     686     27
    1816   116.000000   138.264494      6   1431     49   16.10%     189     27
    1916   116.000000   138.264494      6   1460     30   16.10%     366     27
Heap usage: 489MB (peak 489MB, 37MB system)
B&B tree size: 10MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
E   1931   118.000000   138.264494      7   1460     36   14.66%       0     27
    2077   118.000000   138.264494      7   1344      1   14.66%       1     27
    2179   118.000000   138.264494      7   1421     17   14.66%     579     27
    2279   118.000000   138.264494      7   1448     49   14.66%     408     28
    2379   118.000000   138.264494      7   1537

[1, 2, 3, 7, 9, 10, 11, 12, 13, 14, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 34, 35, 37, 38, 42, 43, 45, 48, 49, 50, 52, 53, 54, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 69, 70, 71, 72, 73, 75, 77, 79, 81, 83, 86, 89, 90, 91, 96, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 110, 111, 115, 116, 118, 120, 121, 122, 123, 124, 127, 128, 130, 134, 138, 143, 144, 147, 148]
FICO Xpress v8.10.0, Hyper, solve started 19:41:23, Aug 15, 2021
Heap usage: 16MB (peak 16MB, 1538KB system)
Maximizing MILP noname
Original problem has:
     45152 rows        22650 cols        90240 elements     22650 globals
Presolved problem has:
      2402 rows         2400 cols         6990 elements      2400 globals
Presolve finished in 0 seconds
Heap usage: 19MB (peak 40MB, 1539KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+

...

   19134   124.000000   139.410280      8   9744     33   11.05%     855     86
   20136   124.000000   139.410280      8  10164     38   11.05%     542     87
   21136   124.000000   139.410280      8  10693     46   11.05%     648     88
   22141   124.000000   139.384232      8  11188     44   11.04%     751     89
   23149   124.000000   139.384232      8  11602     37   11.04%     650     90
   24150   124.000000   139.353661      8  12106     36   11.02%     752     91
   25150   124.000000   139.304288      8  12564     24   10.99%     727     92
   26151   124.000000   139.304288      8  13057     39   10.99%     739     93
   27152   124.000000   139.304288      8  13474     31   10.99%     584     94
   28155   124.000000   139.300448      8  13968     30   10.98%     547     95
   29156   124.000000   139.300448      8  14382     33   10.98%     830     96
   30158   124.000000   139.269202      8  14874     17   10.96%     489     97
   31160   124.000000   139.269202      

[]
FICO Xpress v8.10.0, Hyper, solve started 20:43:20, Aug 15, 2021
Heap usage: 16MB (peak 16MB, 1726KB system)
Maximizing MILP noname
Original problem has:
     45152 rows        22650 cols        90151 elements     22650 globals
Presolved problem has:
      2401 rows         2400 cols         6900 elements      2400 globals
Presolve finished in 0 seconds
Heap usage: 19MB (peak 40MB, 1728KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 59.6GB
 *** Heuristic solution found:    90.000000      Time: 0 ***
Starting concurrent solve with dual, primal and barrier (30 threads)

                           Concurrent-Solve,   0s
    

   1  K    120.000000   141.420063      5      0      0   15.15%    1466     31
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 9
Cut elements in the matrix : 16396
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to 64 tasks.
Heap usage: 24MB (peak 583MB, 3191KB system)
Heap usage: 24MB (peak 583MB, 3191KB system)
B&B tree size: 94k total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    3078   120.000000   141.420063      5      2      1   15.15%    1466     34
    3079   120.000000   141.420063      5      2      3   15.15%    1262     34
    3080   120.000000   141.420063      5      2      3   15.15%    1417     35
    3081   120.000000   141.420063      5      3      4   15.15%    1262     35
    3082   120.000000   141.420063      5      4      4   15.15%    1417     35
    3083   120.000000   141.420063      5      5      5   15.15%    1250     35
    3084   120.000000   141.4200

   10164   124.000000   141.420063      7   6577     37   12.32%     761     84
   10268   124.000000   141.420063      7   6721     57   12.32%     729     84
   10368   124.000000   141.420063      7   6841     20   12.32%     633     84
   10471   124.000000   141.420063      7   6965     30   12.32%    1084     84
   10574   124.000000   141.420063      7   6981     44   12.32%     723     85
   10675   124.000000   141.420063      7   7023     30   12.32%     598     85
   10775   124.000000   141.420063      7   7144     40   12.32%     789     85
   10877   124.000000   141.420063      7   7213     32   12.32%     649     85
   10978   124.000000   141.420063      7   7290     66   12.32%     838     85
   11083   124.000000   141.420063      7   7327     35   12.32%     647     85
Heap usage: 967MB (peak 967MB, 18MB system)
B&B tree size: 18MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   11183   124.000000   141.420063      7   7440

   77289   124.000000   136.963826      7  51300     40    9.47%     812    166
   78292   124.000000   136.851503      7  51949     43    9.39%     483    167
   79295   124.000000   136.851503      7  52487     33    9.39%     699    168
   80298   124.000000   136.851503      7  53045     26    9.39%     570    169
   81299   124.000000   136.851503      7  53573     29    9.39%     687    170
   82299   124.000000   136.766438      7  54209     30    9.33%     650    172
   83305   124.000000   136.766438      7  54788     66    9.33%     649    173
   84305   124.000000   136.234096      7  55380     37    8.98%     906    174
   85308   124.000000   136.169161      7  55892     33    8.94%     604    175
   86313   124.000000   136.169161      7  56531     57    8.94%     601    176
   87313   124.000000   136.169161      7  57093     36    8.94%     491    177
   88315   124.000000   136.122684      7  57662     22    8.91%     664    179
   89320   124.000000   136.122684      